In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:

import os 
import random
import numpy as np
from tqdm import tqdm
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import shutil

import torch 
from torch import nn 
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


# %matplotlib inline
# %config InlineBackend.figure_format='retina'

# sns.set(style='whitegrid', palette='muted', font_scale=1.2)

# HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

# sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

# rcParams['figure.figsize'] = 12, 8

In [3]:
# pwd

In [4]:
# import tensorflow as tf
# print(tf.test.is_gpu_available())

In [5]:
# ! pip install seaborn

In [7]:
# root_dir = "C:\\Users\\yohan\\Desktop\\FYPCodes\\LSTM"
main_dir = "/content/gdrive/MyDrive/FYP/key_arrays_npz"
data_dir = os.path.join(main_dir,"test_key_arrays_npz")
refined_data = os.path.join(main_dir,"Datasets","sequence_data")
class_names = os.listdir(data_dir)
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 1 - train_ratio - val_ratio
batch_size = 32

FileNotFoundError: ignored

In [ ]:
#shutil.rmtree(refined_data)

In [ ]:
file_list = []

for class_n in os.listdir(data_dir):
    file_list += [os.path.join(data_dir,class_n,x) for x in os.listdir(os.path.join(data_dir,class_n))]

In [ ]:
len(file_list)

408

In [ ]:
def break_into_time_frames(coords,vid_size,time_window=50,stride=40):
    shape = coords.shape
    [h,w] = vid_size
    
    #[x,y,z] is the format
    if shape[0]<time_window:
        return None
    
    frame_points = list(range(0,shape[0]-time_window,stride))
    arrays = []
    for i in frame_points:
        arrays.append(coords[i:i+time_window])
    return arrays

def save_arrays(data_dir,file_name,class_name,arrays,vid_size):
    for __id,each_array in enumerate(arrays):
        np.savez(os.path.join(data_dir,f"{class_name}_cls_{file_name}_{__id}.npz"),coords=each_array,video_size=vid_size)


def classname_id(class_name_list):
    id2classname = {k:v for k, v in zip(list(range(len(class_name_list))),class_name_list)}
    classname2id = {v:k for k, v in id2classname.items()}
    return id2classname, classname2id

In [ ]:
gen_required = False

In [ ]:
if gen_required:
    os.makedirs(refined_data,exist_ok=True)
    for each_file in tqdm(file_list):
        a_file = np.load(each_file)
        coords, vid_size = a_file["coords"],a_file["video_size"]
        f_point = break_into_time_frames(coords,vid_size)

        path_parts = each_file.strip().split("\\")
        # print(path_parts)
        class_n = path_parts[-2]
        file_id = path_parts[-1].split(".")[0]

        #print(class_n, file_id)
        if f_point:
            save_arrays(refined_data,file_id,class_n,f_point,vid_size)

In [ ]:
len(os.listdir(refined_data)),np.load(f"{refined_data}\\0_cls_key_array_amani_back_1_0.npz")["coords"].shape

(898, (50, 33, 5))

In [ ]:
data_dir = refined_data

In [ ]:
id2clsname, clsname2id = classname_id(class_names)

In [ ]:
train_file_list = []
val_file_list = []
test_file_list = []

file_list = [os.path.join(data_dir,x) for x in os.listdir(data_dir)]

random.shuffle(file_list)
num_list = len(file_list)

train_range = [0,int(num_list*train_ratio)]
val_range = [int(num_list*train_ratio),int(num_list*(train_ratio+val_ratio))]
test_range = [int(num_list*(train_ratio+val_ratio)),num_list-1]

train_file_list += file_list[train_range[0]:train_range[1]]
val_file_list += file_list[val_range[0]:val_range[1]]
test_file_list += file_list[test_range[0]:test_range[1]]

In [ ]:
len(train_file_list),len(val_file_list),len(test_file_list)

(628, 135, 134)

In [ ]:
# train_file_list = train_file_list[:(len(train_file_list)//batch_size)*batch_size]
# val_file_list = val_file_list[:(len(val_file_list)//batch_size)*batch_size]
# test_file_list = test_file_list[:(len(test_file_list)//batch_size)*batch_size]

In [ ]:
len(train_file_list),len(val_file_list),len(test_file_list)

(608, 128, 128)

In [ ]:
# train_file_list

In [ ]:
class SkeletonDataset(Dataset):
    def __init__(self, file_list,class2id, transform=None, target_transform=None):
        self.file_list = file_list
        self.transform = transform
        self.class2id = class2id
        self.target_transform = target_transform

    

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        a_file = np.load(self.file_list[idx])
        action_type = self.file_list[idx].strip().split("\\")[-1].split("_cls_")[0]
        coords, vid_size = a_file["coords"],a_file["video_size"]
        
        shape = coords.shape
        
        coords = torch.from_numpy(coords).float()
        
        coords = torch.reshape(coords, (shape[0], shape[1]*shape[2]))
        label = torch.clone(coords)
        
        if self.transform:
            coords = self.transform(coords)
        if self.target_transform:
            label = self.target_transform(coords)
        return coords, label, self.class2id[action_type]

In [ ]:
train_data = SkeletonDataset(train_file_list,clsname2id)
val_data = SkeletonDataset(val_file_list,clsname2id)
test_data = SkeletonDataset(test_file_list,clsname2id)

In [ ]:
train_data

In [ ]:
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=False)
val_dl = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
class BiLSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size,device=None, num_layers = 1):
        super(BiLSTMEncoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if device:
          self.device = device
        else:
          self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # define LSTM layer
        self.linear1 = nn.Linear(self.input_size, 128)
        self.linear2 = nn.Linear(128, 256)
        self.linear3 = nn.Linear(256, 512)
        self.lstm = nn.LSTM(input_size = 512, hidden_size = self.hidden_size,
                            num_layers = self.num_layers, bidirectional=True,
                            batch_first=True)
        
        

    def forward(self, x_input):
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence; hidden gives the hidden state and cell state for the last element in the sequence                         
        '''
        
        x = self.linear1(x_input)
        x = self.linear2(x)
        x = self.linear3(x)
        
        lstm_out, self.hidden = self.lstm(x)
        hidden_transformed = torch.cat(self.hidden,0)
        hidden_transformed = torch.transpose(hidden_transformed,0,1)
        hidden_transformed = torch.flatten(hidden_transformed,start_dim=1)
        
        return lstm_out, hidden_transformed.to(self.device)
    
class BiLSTMDecoder(nn.Module):
    def __init__(self, input_size, hidden_size,device=None, num_layers = 1):
        super(BiLSTMDecoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if device:
          self.device = device
        else:
          self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # define LSTM layer
        self.linear1 = nn.Linear(128, self.input_size)
        self.linear2 = nn.Linear(256, 128)
        self.linear3 = nn.Linear(512, 256)
        self.linear4 = nn.Linear(1024, 512)
        
        self.lstm = nn.LSTM(input_size = 512, hidden_size = self.hidden_size,
                            num_layers = self.num_layers, bidirectional=True,
                            batch_first=True)
        
        

    def forward(self,encoder_hidden):
        '''
        : param x_input:               input of shape (seq_len, # in batch, input_size)
        : return lstm_out, hidden:     lstm_out gives all the hidden states in the sequence; hidden gives the hidden state and cell state for the last element in the sequence                         
        '''
        
        hidden_shape = encoder_hidden.shape
        
        hidden = encoder_hidden.view((hidden_shape[0],4,512))
        hidden = torch.transpose(hidden,1,0)
        h1,h2,c1,c2 = torch.unbind(hidden,0)
        h,c = torch.stack((h1,h2)),torch.stack((c1,c2))
        
        dummy_input = torch.rand((32,50,512), requires_grad=True)
        
        lstm_out, self.hidden = self.lstm(dummy_input,(h,c))
        x = self.linear4(lstm_out)
        x = self.linear3(x)
        x = self.linear2(x)
        x = self.linear1(x)
        
        return x

class BiLSTMEncDecModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(BiLSTMEncDecModel, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.encoder = BiLSTMEncoder(input_size,hidden_size,self.device)
        self.decoder = BiLSTMDecoder(input_size,hidden_size,self.device)
        
    def forward(self,x):
        lstm_out,embedding = self.encoder(x)
        embedding = embedding.to(self.device)
        decoder_out = self.decoder(embedding)
        
        return decoder_out
        

        
        
        
        
        
        

In [ ]:
encoder = BiLSTMEncoder(165,512)
decoder = BiLSTMDecoder(165,512)

bilstm_model = BiLSTMEncDecModel(165,512)

165
165


In [ ]:
lstm_out, embedding = encoder(torch.randn((32, 50, 165)))

In [ ]:
decoder_out = decoder(embedding)

In [ ]:
model_out = bilstm_model(torch.randn((32, 50, 165)))
model_out.shape

torch.Size([32, 50, 165])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, train_dataset, val_dataset, n_epochs):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    std_loss = nn.L1Loss(reduction='sum').to(device)
    #contrastive_loss = SupConLoss(contrast_mode="one").to(device)
    history = dict(train=[], val=[])
    model.to(device)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 10000.0
  
    for epoch in range(1, n_epochs + 1):
        model = model.train()

        train_losses = []
        for in_seq,tar_seq,action in tqdm(train_dataset):
            optimizer.zero_grad()
            
            in_seq = in_seq.to(device)
            tar_seq = tar_seq.to(device)
            # print(in_seq.shape)
            seq_pred = model(in_seq)
            
            loss = std_loss(seq_pred, tar_seq)
            #loss += 0.5*contrastive_loss(embed,labels=sample_label.view(-1))
            #print(contrastive_loss(embed,labels=sample_label.view(-1)))

            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        val_losses = []
        model = model.eval()
        with torch.no_grad():
            for in_seq,tar_seq,action in val_dataset:

                in_seq = in_seq.to(device)
                tar_seq = tar_seq.to(device)
                seq_pred = model(in_seq)

                loss = std_loss(seq_pred, tar_seq)
                #loss += 0.5*contrastive_loss(embed,labels=sample_label.view(-1))
                val_losses.append(loss.item())

        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)

        history['train'].append(train_loss)
        history['val'].append(val_loss)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')

    model.load_state_dict(best_model_wts)
    return model.eval(), history

In [ ]:
model, history = train_model(
  bilstm_model, 
  train_dl, 
  val_dl, 
  n_epochs=500
)

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [11:36<00:00, 36.68s/it]


Epoch 1: train loss 109501.10793585527 val loss 75158.12109375


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [11:40<00:00, 36.87s/it]


Epoch 2: train loss 60833.11287006579 val loss 60649.8564453125


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [12:07<00:00, 38.28s/it]


...

Epoch 4: train loss 26256.50699013158 val loss 28766.04052734375


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [11:55<00:00, 37.64s/it]


Epoch 5: train loss 26216.98365542763 val loss 29450.6748046875


 16%|█████████████                                                                      | 3/19 [02:37<13:58, 52.40s/it]


KeyboardInterrupt: 

In [ ]:
import joblib

joblib.dump(bilstm_model,'model\\bilstm_model_1.joblib')

['model\\bilstm_model_1.joblib']

In [ ]:
bilstm = joblib.load('model\\bilstm_model_1.joblib')

In [ ]:
for in_seq,tar_seq,action in tqdm(train_dl):
    in_seq = in_seq.to(device)
    tar_seq = tar_seq.to(device)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 10.16it/s]


In [ ]:
x = np.load('C:\\Users\\yohan\\Desktop\\FYPData\\key_arrays_npz\\test_key_arrays_npz_all\\key_array_amani_back_1.npz')["coords"]

In [ ]:
x.shape

(118, 33, 5)

In [ ]:
action.shape

torch.Size([32])

In [ ]:
tar_seq[0][0].shape

torch.Size([165])

In [ ]:
in_seq.shape

torch.Size([32, 50, 165])

In [ ]:
lstm_out, embedding = encoder(in_seq[0])

In [ ]:
lstm_out.shape

torch.Size([50, 1024])

In [ ]:
embedding.shape

torch.Size([512, 4])

In [ ]:
l = bilstm_model(in_seq)

In [ ]:
l.shape

torch.Size([32, 50, 165])

In [ ]:
ax = plt.figure().gca()

ax.plot(history['train'])
ax.plot(history['val'])
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'])
plt.title('Loss over training epochs')
plt.show();